<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl_2/blob/main/ML_Neumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt


Después de cargar la base de Kaggle en Drive, y después de montar esta cuenta de Drive en Colab, tenemos acceso a las imágenes.

Iniciamos el tratamiento de los datos haciendo un resize() para "homogeneizar" los tamaños de las imágenes.

In [3]:
from os import listdir

def loadImages(path):
    # return array of images

    imagesList = listdir(path)
    loadedImages = []
    for image in imagesList:
        img = cv2.imread(path + image)
        loadedImages.append(cv2.resize(img,(300,300)))

    return np.array(loadedImages)

path = "/content/drive/MyDrive/test/NORMAL/"

# your images in an array
test_normales = loadImages("/content/drive/MyDrive/test/NORMAL/")
test_neumonia= loadImages("/content/drive/MyDrive/test/PNEUMONIA/")
train_normales = loadImages("/content/drive/MyDrive/train/NORMAL/")
train_neumonia= loadImages("/content/drive/MyDrive/train/PNEUMONIA/")
val_normales = loadImages("/content/drive/MyDrive/val/NORMAL/")
val_neumonia= loadImages("/content/drive/MyDrive/val/PNEUMONIA/")


Concatenamos las imágenes de cada clase.

In [4]:
full_test=np.concatenate((test_normales, test_neumonia), axis=0)
full_train=np.concatenate((train_normales, train_neumonia), axis=0)
full_val=np.concatenate((val_normales, val_neumonia), axis=0)


In [10]:
from keras import layers
from keras import models
convNN = models.Sequential()

Definimos las etiquetas de cada clase que se usarán en el entrenamiento de la red.

In [5]:
from keras.utils import to_categorical
train_labels = to_categorical(np.repeat((0,1),(len(train_normales),len(train_neumonia))))
test_labels = to_categorical(np.repeat((0,1),(len(test_normales),len(test_neumonia))))
val_labels = to_categorical(np.repeat((0,1),(len(val_normales),len(val_neumonia))))


Utilizamos una función para describir la composición de una Red.

In [6]:
def resumen(model=None):
    '''
    '''
    header = '{:4} {:16} {:24} {:24} {:10}'.format('#', 'Layer Name','Layer Input Shape','Layer Output Shape','Parameters'
    )
    print('='*(len(header)))
    print(header)
    print('='*(len(header)))
    count=0
    count_trainable=0
    for i, layer in enumerate(model.layers):
        count_trainable += layer.count_params() if layer.trainable else 0
        input_shape = '{}'.format(layer.input_shape)
        output_shape = '{}'.format(layer.output_shape)
        str = '{:<4d} {:16} {:24} {:24} {:10}'.format(i,layer.name, input_shape, output_shape, layer.count_params())
        print(str)
        count += layer.count_params()
    print('_'*(len(header)))
    print('Total Parameters : ', count)
    print('Total Trainable Parameters : ', count_trainable)
    print('Total No-Trainable Parameters : ', count-count_trainable)
    

En este trabajo utilizaremos *EfficientNetB3* por tener una cantidad "pequeña" de parámetros.

In [7]:
from keras.applications import EfficientNetB3 

Efficient = EfficientNetB3 (weights='imagenet',
                  include_top=True,
                  input_shape=(300, 300, 3))

resumen(Efficient)

#    Layer Name       Layer Input Shape        Layer Output Shape       Parameters
0    input_1          [(None, 300, 300, 3)]    [(None, 300, 300, 3)]             0
1    rescaling        (None, 300, 300, 3)      (None, 300, 300, 3)               0
2    normalization    (None, 300, 300, 3)      (None, 300, 300, 3)               7
3    tf.math.truediv  (None, 300, 300, 3)      (None, 300, 300, 3)               0
4    stem_conv_pad    (None, 300, 300, 3)      (None, 301, 301, 3)               0
5    stem_conv        (None, 301, 301, 3)      (None, 150, 150, 40)           1080
6    stem_bn          (None, 150, 150, 40)     (None, 150, 150, 40)            160
7    stem_activation  (None, 150, 150, 40)     (None, 150, 150, 40)              0
8    block1a_dwconv   (None, 150, 150, 40)     (None, 150, 150, 40)            360
9    block1a_bn       (None, 150, 150, 40)     (None, 150, 150, 40)            160
10   block1a_activation (None, 150, 150, 40)     (None, 150, 150, 40)              0
11

Observamos que hay 12 millones de parámetros, y la red recibe un input de dimensiones (300,300,3). I.e. una imagen de 300$\times$300 con 3 canales.

Definimos lo que será nuestra parte convolucional, definida por las primeras capas de EfficientNetB3.

In [8]:
Efficient=None 
if Efficient != None:
    del Efficient
    
from keras.applications import EfficientNetB3

conv_base = EfficientNetB3(weights='imagenet',
                  include_top=False,
                  input_shape=(300, 300, 3))
    
resumen(conv_base)   

#    Layer Name       Layer Input Shape        Layer Output Shape       Parameters
0    input_2          [(None, 300, 300, 3)]    [(None, 300, 300, 3)]             0
1    rescaling_1      (None, 300, 300, 3)      (None, 300, 300, 3)               0
2    normalization_1  (None, 300, 300, 3)      (None, 300, 300, 3)               7
3    tf.math.truediv_1 (None, 300, 300, 3)      (None, 300, 300, 3)               0
4    stem_conv_pad    (None, 300, 300, 3)      (None, 301, 301, 3)               0
5    stem_conv        (None, 301, 301, 3)      (None, 150, 150, 40)           1080
6    stem_bn          (None, 150, 150, 40)     (None, 150, 150, 40)            160
7    stem_activation  (None, 150, 150, 40)     (None, 150, 150, 40)              0
8    block1a_dwconv   (None, 150, 150, 40)     (None, 150, 150, 40)            360
9    block1a_bn       (None, 150, 150, 40)     (None, 150, 150, 40)            160
10   block1a_activation (None, 150, 150, 40)     (None, 150, 150, 40)              0
1

Esta parte tiene 10 millones de parámetros, y tiene un output de (10$\times$10$\times$1536).

A continuación se describe brevemente el procedimiento de entrenamiento.



*   Congelamos las capas convolucionales heredadas por EfficientnetB3. De esta manera, los pesos de esta red se mantienen fijos, pues la transferencia de conocimientos supone que estos pesos extraen las características de las imagenes que se ingresan.
*   Agregamos capas entranables a nuestro modelo. Agregamos capas convolucionales y densas, en este caso la primera capa debe recibir como input una imagen que coincida con la capa de salida de la parte convolucional heredada. Como se mencionó antes, de (10$\times$10$\times$1536). Pero esto lo hace automáticamente keras.
*   Hasta ahora, tenemos una cantidad mucho menor de parámetros, que corresponden a los pesos de las capas que sí son entrenables. 
*   Procedemos a entrenar la red. De hecho, como las capas iniciales siguen congeladas, los únicos pesos que se modificarán, serán correspondientes a las capas que agregamos al final.

Lo que quiero que haga:
Cargo los pesos de la parte convolucional de una Red existente.
Congelo estas capas y pesos para que no se entrenen.
Activo la capa de input y agrego capas densas al final de esta convolucional que sí son entrenables.
Entreno a la red entera (solo las ultimas capas son entreables), pasandole mis imagenes con sus etiquetas (imagenes,etiquetas) y obtengo pesos para las capas densas.


In [31]:
model = models.Sequential()

conv_base.trainable = False

model.add(conv_base)   


In [32]:
model.add(layers.MaxPooling2D(pool_size=(2, 2)))#4/2
model.add(layers.Conv2D(filters     = 64, 
                        kernel_size = (3, 3), 
                        activation  = 'relu' ))
model.add(layers.Flatten())
model.add(layers.Dense(256, 
                        activation='relu'))
model.add(layers.Dense(64, 
                        activation='relu'))
model.add(layers.Dense(2, 
                        activation='softmax'))

In [33]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 10, 10, 1536)     10783535  
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 1536)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 3, 3, 64)          884800    
                                                                 
 flatten_6 (Flatten)         (None, 576)               0         
                                                                 
 dense_18 (Dense)            (None, 256)               147712    
                                                                 
 dense_19 (Dense)            (None, 64)                16448     
                                                      

In [34]:
model.compile(loss='binary_crossentropy',
              optimizer="nadam",
              metrics=['acc'])

In [35]:
import time
tic = time.time()

model.fit(x = full_train, 
           y = train_labels, 
           epochs    =3,
           batch_size=200,
           verbose=2)

print('seconds=', time.time()-tic)

Epoch 1/3
27/27 - 59s - loss: 0.4309 - acc: 0.8401 - 59s/epoch - 2s/step
Epoch 2/3
27/27 - 33s - loss: 0.1348 - acc: 0.9477 - 33s/epoch - 1s/step
Epoch 3/3
27/27 - 33s - loss: 0.0850 - acc: 0.9701 - 33s/epoch - 1s/step
seconds= 126.66766333580017


Como vemos, tenemos un muy buen desempeño en muy poco tiempo, aprovechando el conocimiento de la red EfficientNetB3.

A continuación, nos gustaría descongelar algunas de las últimas capas heredadas de EfficientnetB3, de manera que se mejore el ajuste.

In [36]:
for layer in conv_base.layers:
    print(layer.name)

input_2
rescaling_1
normalization_1
tf.math.truediv_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block1b_dwconv
block1b_bn
block1b_activation
block1b_se_squeeze
block1b_se_reshape
block1b_se_reduce
block1b_se_expand
block1b_se_excite
block1b_project_conv
block1b_project_bn
block1b_drop
block1b_add
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block

Elegiré unas cuantas de las capas del último bloque.

In [47]:
resumen(model)   

#    Layer Name       Layer Input Shape        Layer Output Shape       Parameters
0    efficientnetb3   (None, 300, 300, 3)      (None, 10, 10, 1536)       10783535
1    max_pooling2d_3  (None, 10, 10, 1536)     (None, 5, 5, 1536)                0
2    conv2d_11        (None, 5, 5, 1536)       (None, 3, 3, 64)             884800
3    flatten_6        (None, 3, 3, 64)         (None, 576)                       0
4    dense_18         (None, 576)              (None, 256)                  147712
5    dense_19         (None, 256)              (None, 64)                    16448
6    dense_20         (None, 64)               (None, 2)                       130
__________________________________________________________________________________
Total Parameters :  11832625
Total Trainable Parameters :  1049090
Total No-Trainable Parameters :  10783535


In [48]:
conv_base.trainable = True
set_trainable = False

for layer in conv_base.layers:
    if layer.name in ['block7a_expand_conv',
'block7a_expand_bn',
'block7a_expand_activation',
'block7a_dwconv',
'block7a_bn',
'block7a_activation',
'block7a_se_squeeze',
'block7a_se_reshape',
'block7a_se_reduce',
'block7a_se_expand',
'block7a_se_excite',
'block7a_project_conv',
'block7a_project_bn',
'block7b_expand_conv',
'block7b_expand_bn',
'block7b_expand_activation',
'block7b_dwconv',
'block7b_bn',
'block7b_activation',
'block7b_se_squeeze',
'block7b_se_reshape',
'block7b_se_reduce',
'block7b_se_expand',
'block7b_se_excite',
'block7b_project_conv',
'block7b_project_bn',
'block7b_drop',
'block7b_add',
'top_conv',
'top_bn',
'top_activation']:
        layer.trainable = True
    else:
        layer.trainable = False
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])
resumen(model)   

#    Layer Name       Layer Input Shape        Layer Output Shape       Parameters
0    efficientnetb3   (None, 300, 300, 3)      (None, 10, 10, 1536)       10783535
1    max_pooling2d_3  (None, 10, 10, 1536)     (None, 5, 5, 1536)                0
2    conv2d_11        (None, 5, 5, 1536)       (None, 3, 3, 64)             884800
3    flatten_6        (None, 3, 3, 64)         (None, 576)                       0
4    dense_18         (None, 576)              (None, 256)                  147712
5    dense_19         (None, 256)              (None, 64)                    16448
6    dense_20         (None, 64)               (None, 2)                       130
__________________________________________________________________________________
Total Parameters :  11832625
Total Trainable Parameters :  1049090
Total No-Trainable Parameters :  10783535
